# YOLO

In [4]:
import sys
import numpy as np
import cv2

model='C:/deep/opencv/model/yolov3.weights'
config='C:/deep/opencv/model/yolov3.cfg'
class_labels='C:/deep/opencv/model/coco.names'
confThreshold=0.5
nmsThreshold=0.4

img_files=['C:/deep/opencv/image/dog.jpg', 'C:/deep/opencv/image/person.jpg',
           'C:/deep/opencv/image/yolo_01.jpg', 'C:/deep/opencv/image/sheep.jpg','C:/deep/opencv/image/kite.jpg']

net=cv2.dnn.readNet(model, config)

if net.empty():
    print('Net open failed!')
    sys.exit()
    
classes=[]
with open(class_labels, 'rt') as f:
    classes=f.read().rstrip('\n').split('\n')
    
colors=np.random.uniform(0, 255, size=(len(classes),3))

layer_names=net.getLayerNames()
output_layers=[layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

#실행
for f in img_files:
    img=cv2.imread(f)
    if img is None:
        continue
        
    #블롭 생선 & 추론
    blob=cv2.dnn.blobFromImage(img, 1/255., (320,320), swapRB=True)
    net.setInput(blob)
    outs=net.forward(output_layers)
    
    h,w=img.shape[:2]
    class_ids=[]
    confidences=[]
    boxes=[]
    
    for out in outs:
        for detection in out:
            scores=detection[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence > confThreshold:
                #바운딩 박스 중심 좌표 & 박스 크기
                cx=int(detection[0]*w)
                cy=int(detection[1]*h)
                bw=int(detection[2]*w)
                bh=int(detection[3]*h)
                #바운딩 박스 좌상단 좌표
                sx=int(cx-bw/2)
                sy=int(cy-bh/2)
                boxes.append([sx, sy, bw, bh])
                confidences.append(float(confidence))
                class_ids.append(int(class_id))
                
    #비최대 억제
    indices=cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i=i[0]
        sx, sy, bw, bh=boxes[i]
        label=f'{classes[class_ids[i]]}: {confidences[i]:.2}'
        color=colors[class_ids[i]]
        cv2.rectangle(img, (sx, sy, bw, bh), color, 2)
        cv2.putText(img, label, (sx,sy-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 1, cv2.LINE_AA)
    
    t, _=net.getPerfProfile()
    label='Inference time: %.2f ms' % (t *1000.0/cv2.getTickFrequency())
    cv2.putText(img, label, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
               (0,0,255),1, cv2.LINE_AA)

    cv2.imshow('img', img)
    cv2.waitKey()
cv2.destroyAllWindows()

# EAST

In [9]:
import sys
import math
import numpy as np
import cv2

def decode(scores, geometry, scoreThreshold):
    detections=[]
    confidences=[]
    height=scores.shape[2]
    width=scores.shape[3]
    
    for y in range(0, height):
        #Extract data from scores
        scoresData=scores[0][0][y]
        x0_data=geometry[0][0][y]
        x1_data=geometry[0][1][y]
        x2_data=geometry[0][2][y]
        x3_data=geometry[0][3][y]
        anglesData=geometry[0][4][y]
        
        for x in range(0, width):
            score=scoresData[x]
            if(score < scoreThreshold):
                continue
            #feature map은 320x320 블롭의 1/4크기이므로, 다시 4배 확대
            offsetX=x*4.0
            offsetY=y*4.0
            angle=anglesData[x]
            #(offsetX, offsetY) 위치에서 회전된 사각형 정보 추출
            cosA=math.cos(angle)
            sinA=math.sin(angle)
            h=x0_data[x]+x2_data[x]
            w=x1_data[x]+x3_data[x]
            #회전된 사각형의 한쪽 모서리 점 좌표 계산
            offset=([offsetX+cosA*x1_data[x]+sinA*x2_data[x],
                    offsetY-sinA*x1_data[x]+cosA*x2_data[x]])
            #회전된 사각형의 대각선에 위치한 두 모서리 점 좌표 계산
            p1=(-sinA*h+offset[0], -cosA*h+offset[1])
            p3=(-cosA*w+offset[0], sinA*w+offset[1])
            center=((p1[0]+p3[0])/2, (p1[1]+p3[1])/2)
            
            detections.append((center, (w,h), -1*angle*180.0/math.pi))
            confidences.append(float(score))
    return [detections, confidences]

#모델&설정 파일
model='C:/deep/opencv/model/frozen_east_text_detection.pb'
confThreshold=0.5
nmsThreshold=0.4

#테스트 이미지 파일
img_files=['C:/deep/opencv/image/road_closed.jpg','C:/deep/opencv/image/patient.jpg','C:/deep/opencv/image/ocr_ex2.jpg']
#네트워크 생성
net=cv2.dnn.readNet(model)
if net.empty():
    print('Net open failed!')
    sys.exit()
#출력 레이어 이름 받아오기
'''
layer_names=net.getLayerNames()
output_layers=[layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(output_layers)
'''
#실행
for f in img_files:
    img=cv2.imread(f)
    if img is None:
        continue
        
    #블롭 생선 & 추론
    blob=cv2.dnn.blobFromImage(img, 1, (320,320), (123.68, 116.78, 103.94),True)
    net.setInput(blob)
    scores, geometry=net.forward(['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'])
    #scores.shape=(1,1,80,80)
    #geonetry.shape=(1,5,80,80)
    
    #score가 confThreshold보다 큰 RBOX 정보를 RotatedRect 형식으로 변환하여 반환
    [boxes, confidences]=decode(scores, geometry, confThreshold)
    #회전된 사각형에 대한 비최대 억제
    indices=cv2.dnn.NMSBoxesRotated(boxes, confidences, confThreshold, nmsThreshold)
    rw=img.shape[1]/320
    rh=img.shape[0]/320
    
    for i in indices:
        #회전된 사각형의 네 모서리 점 좌표 계산 & 표시
        vertices=cv2.boxPoints(boxes[i[0]])
        
        for j in range(4):
            vertices[j][0] *= rw
            vertices[j][1] *= rh
            
        for j in range(4):
            p1=(vertices[j][0], vertices[j][1])
            p2=(vertices[(j+1)%4][0], vertices[(j+1)%4][1])
            cv2.line(img, p1, p2, (0,0,255), 2, cv2.LINE_AA)
            
    cv2.imshow('img',img)
    cv2.waitKey()
    
cv2.destroyAllWindows()

# 가격표 EAST & OCR

In [ ]:
#원하는 이미지 잘라내기

import sys
import numpy as np
import cv2
import pytesseract
import math

#모델&설정 파일
model='C:/deep/opencv/model/frozen_east_text_detection.pb'
confThreshold=0.5
nmsThreshold=0.4

#가격표 좌표로 전처리
src=cv2.imread('C:/deep/opencv/image/ocr_ex2.jpg')

if src is None:
    print('image load failed')
    sys.exit()

w,h=600,480
srcQuad=np.array([[353, 494],[762, 486],[772, 738],[351, 747]], np.float32)
dstQuad=np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]], np.float32)

pers=cv2.getPerspectiveTransform(srcQuad, dstQuad)
dst=cv2.warpPerspective(src,pers,(w,h))

#테스트 이미지 파일
img_files=['C:/deep/opencv/image/road_closed.jpg','C:/deep/opencv/image/patient.jpg','C:/deep/opencv/image/ocr_ex2.jpg']
#네트워크 생성
net=cv2.dnn.readNet(model)
if net.empty():
    print('Net open failed!')
    sys.exit()
#출력 레이어 이름 받아오기
'''
layer_names=net.getLayerNames()
output_layers=[layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(output_layers)
'''
#실행
for f in img_files:
    img=cv2.imread(f)
    if img is None:
        continue
        
    #블롭 생선 & 추론
    blob=cv2.dnn.blobFromImage(img, 1, (320,320), (123.68, 116.78, 103.94),True)
    net.setInput(blob)
    scores, geometry=net.forward(['feature_fusion/Conv_7/Sigmoid', 'feature_fusion/concat_3'])
    #scores.shape=(1,1,80,80)
    #geonetry.shape=(1,5,80,80)
    
    #score가 confThreshold보다 큰 RBOX 정보를 RotatedRect 형식으로 변환하여 반환
    [boxes, confidences]=decode(scores, geometry, confThreshold)
    #회전된 사각형에 대한 비최대 억제
    indices=cv2.dnn.NMSBoxesRotated(boxes, confidences, confThreshold, nmsThreshold)
    rw=img.shape[1]/320
    rh=img.shape[0]/320
    
    for i in indices:
        #회전된 사각형의 네 모서리 점 좌표 계산 & 표시
        vertices=cv2.boxPoints(boxes[i[0]])
        
        for j in range(4):
            vertices[j][0] *= rw
            vertices[j][1] *= rh
            
        for j in range(4):
            p1=(vertices[j][0], vertices[j][1])
            p2=(vertices[(j+1)%4][0], vertices[(j+1)%4][1])
            cv2.line(img, p1, p2, (0,0,255), 2, cv2.LINE_AA)
            
    cv2.imshow('img',img)
    cv2.waitKey()
    
cv2.destroyAllWindows()


#tesseract-ocr
print(pytesseract.image_to_string(dst, lang='Hangul+eng'))
img_gray=cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

cv2.imshow('src', src)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()


In [59]:
#객체 좌표 찾기
import sys
import numpy as np
import cv2

oldx=oldy=-1

def on_mouse(event, x,y, flags, param):
    global oldx, oldy
    if event == cv2.EVENT_LBUTTONDOWN:
        oldx, oldy=x,y
        print('EVENT_LBUTTONDOWN: %d, %d' % (x,y))

src=cv2.imread('C:/deep/opencv/image/ocr_ex12.jpg')

cv2.namedWindow('image')
cv2.setMouseCallback('image', on_mouse, src)


cv2.imshow('image', src)
cv2.waitKey()
cv2.destroyAllWindows()

EVENT_LBUTTONDOWN: 146, 237
EVENT_LBUTTONDOWN: 547, 224
EVENT_LBUTTONDOWN: 514, 437
EVENT_LBUTTONDOWN: 191, 460


In [61]:
#원하는 이미지 잘라내기

import sys
import numpy as np
import cv2
import pytesseract

src=cv2.imread('C:/deep/opencv/image/ocr_ex12.jpg')

if src is None:
    print('image load failed')
    sys.exit()

w,h=600,480
srcQuad=np.array([[146, 237],[547, 224],[514, 437],[191, 460]], np.float32)
dstQuad=np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]], np.float32)

pers=cv2.getPerspectiveTransform(srcQuad, dstQuad)
dst=cv2.warpPerspective(src,pers,(w,h))
img_gray=cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)

#tesseract-ocr
print(pytesseract.image_to_string(img_gray, lang='Hangul+eng'))


cv2.imshow('src', src)
cv2.imshow('dst',img_gray)
cv2.waitKey()
cv2.destroyAllWindows()


# pdf참조

In [16]:
import sys
import numpy as np
import cv2
import pytesseract

#좌측상단부터 반시계방향으로 0,1,2,3번 좌표
def reorderPts(pts):
    idx=np.lexsort((pts[:,1], pts[:, 0 ]))# 칼럼 0 --> 칼럼 1 순으로 정렬한 인덱스를 반환
    pts=pts[idx] # x 좌표로 정렬(idx순서대로 정렬)
    if pts[0, 1]> pts[1, 1]:
        pts[[0,1]]=pts[[1,0]]

    if pts[2, 1]< pts[3, 1]:
        pts[[2,3]]=pts[[3,2]]
    return pts

#filename='ocr_ex4.jpg'

#if len(sys.argv)> 1:
#    filename=sys.argv[1]

src=cv2.imread('C:/deep/opencv/image/unnamed4.jpg')

dw, dh= 720, 400
srcQuad=np.array([[0,0],[0,0],[0,0],[0,0]],np.float32)
dstQuad=np.array([[0,0],[0,dh],[dw,dh],[dw,0]], np.float32)#좌측상단부터 반시계방향
dst=np.zeros((dh,dw), np.uint8)

src_gray=cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
th, src_bin=cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#외곽선 검출
contours, _ =cv2.findContours(src_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

for pts in contours:
    #너무 작은 객체 제외
    if cv2.contourArea(pts) < 1000:
        continue
    #외곽선 근사화
    approx=cv2.approxPolyDP(pts, cv2.arcLength(pts, True)* 0.02, True)
    #컨벡스가 아니면 제외
    if not cv2.isContourConvex(approx) or len(approx) != 4:
        continue
    srcQuad=reorderPts(approx.reshape(4, 2).astype(np.float32))
    pers=cv2.getPerspectiveTransform(srcQuad, dstQuad)
    dst=cv2.warpPerspective(src, pers, (dw, dh), flags=cv2.INTER_CUBIC)
    dst_rgb=cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
    print(pytesseract.image_to_string(dst_rgb, lang='Hangul+eng+kor'))

cv2.imshow('src', src)
cv2.imshow('dst_rgb', dst_rgb)
cv2.waitKey()
cv2.destroyAllWindows()